# retrieve run data from mongo

In [1]:
from pymongo import MongoClient
import polars as pl

client = MongoClient("localhost", 27017)
db = client["trader"]
collection = db["message"]
trading_session_id = "3ff39a04-d244-44f1-b9a0-a33465394143"
documents = list(collection.find({"trading_session_id": trading_session_id}))
run_data = pl.DataFrame(documents)

In [25]:
for i in range(6):

    print(run_data['content'][i]['active_orders'])


[]
[{'id': b'\x85k>\x17).E\xd8\x96\xff{9K\xe9\xe8\x9d', 'trader_id': 'NOISE_457039e4-7048-4c07-a8ea-d879531dcf3f', 'order_type': 1, 'amount': 1.0, 'price': 2000.0, 'timestamp': datetime.datetime(2024, 5, 15, 14, 5, 26, 676000)}, {'id': b'\x12\x83\xf5k7-N\x8d\xb43F\xe9\x1fce\x93', 'trader_id': 'NOISE_457039e4-7048-4c07-a8ea-d879531dcf3f', 'order_type': 1, 'amount': 1.0, 'price': 1999.0, 'timestamp': datetime.datetime(2024, 5, 15, 14, 5, 27, 55000)}]
[{'id': b'\x85k>\x17).E\xd8\x96\xff{9K\xe9\xe8\x9d', 'trader_id': 'NOISE_457039e4-7048-4c07-a8ea-d879531dcf3f', 'order_type': 1, 'amount': 1.0, 'price': 2000.0, 'timestamp': datetime.datetime(2024, 5, 15, 14, 5, 26, 676000)}, {'id': b'\x12\x83\xf5k7-N\x8d\xb43F\xe9\x1fce\x93', 'trader_id': 'NOISE_457039e4-7048-4c07-a8ea-d879531dcf3f', 'order_type': 1, 'amount': 1.0, 'price': 1999.0, 'timestamp': datetime.datetime(2024, 5, 15, 14, 5, 27, 55000)}]
[{'id': b'\x85k>\x17).E\xd8\x96\xff{9K\xe9\xe8\x9d', 'trader_id': 'NOISE_457039e4-7048-4c07-a8ea-

In [27]:
run_data['content'][5]['active_orders']

[{'id': b'\x12\x83\xf5k7-N\x8d\xb43F\xe9\x1fce\x93',
  'trader_id': 'NOISE_457039e4-7048-4c07-a8ea-d879531dcf3f',
  'order_type': 1,
  'amount': 1.0,
  'price': 1999.0,
  'timestamp': datetime.datetime(2024, 5, 15, 14, 5, 27, 55000)},
 {'id': b" \xfft\x88\x90pC\xb8\xb6\xc7\xa5'%o\xa3\xef",
  'trader_id': 'NOISE_457039e4-7048-4c07-a8ea-d879531dcf3f',
  'order_type': 1,
  'amount': 1.0,
  'price': 1995.0,
  'timestamp': datetime.datetime(2024, 5, 15, 14, 5, 29, 893000)},
 {'id': b'\xb7\xa36\xe1\xfb`G\xb4\xbd\x10\xb1\xccQ\xba\xd2\x16',
  'trader_id': 'NOISE_457039e4-7048-4c07-a8ea-d879531dcf3f',
  'order_type': -1,
  'amount': 1.0,
  'price': 2001.0,
  'timestamp': datetime.datetime(2024, 5, 15, 14, 5, 32, 855000)}]

# retrieve processed lobster data from duckdb 

In [3]:
from analysis.run_lobster import pick_one

lobster_data = pick_one('3ff39a04-d244-44f1-b9a0-a33465394143')

/Users/venvoo/Library/CloudStorage/GoogleDrive-venvoooo@gmail.com/My Drive/trader/trader_london/analysis/prepare_data.py:49: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  .str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S.%f", strict=False)


In [21]:
lobster_data.drop(['trading_session_id', 'Event Type', 'Order ID', 'Size', 'Price',
       'Direction'], inplace=True, axis=1)

In [24]:
lobster_data.head(6)

,LOBSTER_BOOK_0,LOBSTER_BOOK_1,LOBSTER_BOOK_2,LOBSTER_BOOK_3,LOBSTER_BOOK_4,LOBSTER_BOOK_5,LOBSTER_BOOK_6,LOBSTER_BOOK_7,LOBSTER_BOOK_8,LOBSTER_BOOK_9,...,LOBSTER_BOOK_30,LOBSTER_BOOK_31,LOBSTER_BOOK_32,LOBSTER_BOOK_33,LOBSTER_BOOK_34,LOBSTER_BOOK_35,LOBSTER_BOOK_36,LOBSTER_BOOK_37,LOBSTER_BOOK_38,LOBSTER_BOOK_39
Time,,,,,,,,,,,,,,,,,,,,,
0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.999633,2000.0,0.0,2000.0,1.0,2001.0,0.0,1999.0,1.0,2002.0,0.0,...,1993.0,0.0,2008.0,0.0,1992.0,0.0,2009.0,0.0,1991.0,0.0
2.999534,2000.0,0.0,2000.0,1.0,2001.0,0.0,1999.0,1.0,2002.0,0.0,...,1993.0,0.0,2008.0,0.0,1992.0,0.0,2009.0,0.0,1991.0,0.0
4.999430,2000.0,0.0,2001.0,1.0,2001.0,0.0,2000.0,1.0,2002.0,0.0,...,1991.0,0.0,2008.0,0.0,1990.0,0.0,2009.0,0.0,1989.0,0.0
6.000148,2000.0,0.0,2000.0,1.0,2001.0,0.0,1999.0,1.0,2002.0,0.0,...,1990.0,0.0,2008.0,0.0,1989.0,0.0,2009.0,0.0,1988.0,0.0
9.000304,2001.0,1.0,1999.0,1.0,2002.0,0.0,1995.0,1.0,2003.0,0.0,...,1989.0,0.0,2009.0,0.0,1988.0,0.0,2010.0,0.0,1987.0,0.0
